In [ ]:
import torch
import numpy
import data
import models
import utils
from matplotlib import pyplot as plt


In [ ]:
torch.cuda.set_device(3)

In [ ]:
train_batches = data.get_loaders("cifar10", -1, batch_size=1, train_set=True, shuffle=False, data_augm=False)
model = models.get_model("cnn", n_cls=10, half_prec=False, shapes_dict=data.shapes_dict["cifar10"], n_filters_cnn=6).cuda()
model_saved = utils.load_model("211654")
model.load_state_dict(model_saved["last"])

In [ ]:
m = torch.nn.Conv2d(3, 1, 3, stride=1, padding=1, bias=False)
relu = torch.nn.ReLU()
m.weight.shape

In [ ]:
for name, parameters in model.named_parameters():
    if name=="_model.0.weight":
        weight = parameters.detach().cpu()
    if name=="_model.0.bias":
        bias = parameters.detach().cpu()

In [ ]:
weight[0].unsqueeze(dim=0)

In [ ]:
bias

In [ ]:
filter_i = 1

In [ ]:
m.weight.data = weight[filter_i].unsqueeze(dim=0)

In [ ]:
m.weight

In [ ]:
delta = torch.zeros((1,3,32,32))
delta.uniform_(-10/255, 10/255)

In [ ]:
for i, (X, y) in enumerate(train_batches):
    if i == 9:
        print(X.shape)
        
        # feature_eta = relu(m(X+delta)+bias[filter_i])
        # feature = relu(m(X))
        feature_eta = m(X+delta)+bias[filter_i]
        feature = m(X)
        print(feature.shape)

        break

In [ ]:
feature

In [ ]:
relu(feature)

In [ ]:
torch.sign(relu(feature))

In [ ]:
norm_feature = numpy.linalg.norm(feature.detach())
norm_feature_eta = numpy.linalg.norm(feature_eta.detach())
print(norm_feature)
print(norm_feature_eta)

In [ ]:
torch.sign(feature).detach().sum()

In [ ]:
torch.sign(relu(feature)).detach().sum()

In [ ]:
torch.sign(feature_eta).detach().sum()

In [ ]:
torch.sign(relu(feature_eta)).detach().sum()

In [ ]:
(torch.sign(relu(feature))*torch.sign(relu(feature_eta))).sum()

In [ ]:
plt.matshow(relu(feature).detach().numpy()[0][0], cmap=plt.cm.Reds)
# plt.matshow(feature.detach().numpy()[0][0], cmap=plt.cm.coolwarm)
plt.axis('off')
plt.colorbar(fraction=0.0455, pad=0.05)
plt.savefig("img/feature_filter{}.pdf".format(filter_i))
# plt.matshow(feature_eta.detach().numpy()[0][0], cmap=plt.cm.Reds)
plt.matshow(feature_eta.detach().numpy()[0][0], cmap=plt.cm.coolwarm)
plt.axis('off')
plt.colorbar(fraction=0.0455, pad=0.05)
plt.savefig("img/feature_eta_filter{}.pdf".format(filter_i))

In [ ]:
from torchvision import transforms
toPIL = transforms.ToPILImage()
pic = toPIL(X[0])
pic.save("X0.jpg")
pic = toPIL((X+delta)[0])
pic.save("X0_eta.jpg")